### Instalación de Hadoop

In [1]:
%%bash
wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
tar -xzf hadoop-3.3.6.tar.gz
mv  hadoop-3.3.6/ /usr/local/

--2024-02-09 08:13:08--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  165K 72m5s
    50K .......... .......... .......... .......... ..........  0%  330K 54m2s
   100K .......... .......... .......... .......... ..........  0% 73.1M 36m4s
   150K .......... .......... .......... .......... ..........  0% 1.25M 29m22s
   200K .......... .......... .......... .......... ..........  0%  439K 28m54s
   250K .......... .......... .......... .......... ..........  0% 29.3M 24m9s
   300K .......... .......... .......... .......... ..........  0% 88.6M 20m43s
   350K .....

Actualizamos variables de entorno

In [2]:
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/hadoop-3.3.6/bin"

Comprobamos instalación

In [3]:
!hadoop version

Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /usr/local/hadoop-3.3.6/share/hadoop/common/hadoop-common-3.3.6.jar


### Instalación de Pig

In [4]:
%%bash
wget https://downloads.apache.org/pig/pig-0.17.0/pig-0.17.0.tar.gz
tar -xzf pig-0.17.0.tar.gz
cp -r pig-0.17.0/ /usr/local/

--2024-02-09 08:14:49--  https://downloads.apache.org/pig/pig-0.17.0/pig-0.17.0.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230606579 (220M) [application/x-gzip]
Saving to: ‘pig-0.17.0.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  155K 24m13s
    50K .......... .......... .......... .......... ..........  0%  312K 18m7s
   100K .......... .......... .......... .......... ..........  0% 19.3M 12m8s
   150K .......... .......... .......... .......... ..........  0% 19.0M 9m9s
   200K .......... .......... .......... .......... ..........  0%  317K 9m41s
   250K .......... .......... .......... .......... ..........  0% 17.3M 8m6s
   300K .......... .......... .......... .......... ..........  0% 21.7M 6m58s
   350K .......... ..........

In [5]:
import os
os.environ["PIG_HOME"] = "/usr/local/pig-0.17.0"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/pig-0.17.0/bin"
os.environ["PIG_CLASSPATH"] = "/usr/local/hadoop-3.3.6/conf"

In [6]:
!pig -h -version


Apache Pig version 0.17.0 (r1797386) 
compiled Jun 02 2017, 15:41:58

USAGE: Pig [options] [-] : Run interactively in grunt shell.
       Pig [options] -e[xecute] cmd [cmd ...] : Run cmd(s).
       Pig [options] [-f[ile]] file : Run cmds found in file.
  options include:
    -4, -log4jconf - Log4j configuration file, overrides log conf
    -b, -brief - Brief logging (no timestamps)
    -c, -check - Syntax check
    -d, -debug - Debug level, INFO is default
    -e, -execute - Commands to execute (within quotes)
    -f, -file - Path to the script to execute
    -g, -embedded - ScriptEngine classname or keyword for the ScriptEngine
    -h, -help - Display this message. You can specify topic to get help for that topic.
        properties is the only topic currently supported: -h properties.
    -i, -version - Display version information
    -l, -logfile - Path to client side log file; default is current working directory.
    -m, -param_file - Path to the parameter file
    -p, -param - K

### Ejecución y tratamiento de datos

1. Bajar el fichero de datos de reclamaciones de seguros

In [7]:
!wget https://raw.githubusercontent.com/Big-Data-IA/Big-Data-Aplicado/main/Tema_5/train.csv

--2024-02-09 08:15:32--  https://raw.githubusercontent.com/Big-Data-IA/Big-Data-Aplicado/main/Tema_5/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4869451 (4.6M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   4.64M  --.-KB/s    in 0.06s   

2024-02-09 08:15:33 (84.0 MB/s) - ‘train.csv’ saved [4869451/4869451]



2. *Lo movemos a HDFS* y comprobamos su estado utilizando `hdfs dfs -head`

In [8]:
%%bash
hdfs dfs -mkdir claims
hdfs dfs -put train.csv claims
hdfs dfs -mv claims/train.csv claims/claims.csv
hdfs dfs -head claims/claims.csv

ClaimNumber,DateTimeOfAccident,DateReported,Age,Gender,MaritalStatus,DependentChildren,DependentsOther,WeeklyWages,PartTimeFullTime,HoursWorkedPerWeek,DaysWorkedPerWeek,ClaimDescription,InitialIncurredCalimsCost,UltimateIncurredClaimCost
WC1840818,1988-01-01 09:00:00,1988-03-07 00:00:00,59,F,M,0,0,334.9,F,40,5,STRUCK KNIFE SLIPPED CUT FINGER LACERATED RIGHT MIDDLE FINGER,2000,1527.825031
WC1887818,1988-01-01 09:00:00,1988-01-21 00:00:00,45,F,S,0,0,48.26,F,31.75,5,STRUCK FALLING DOOR PULLED MUSCLE IN FOREARM,2000,49711.11598
WC1693016,1988-01-01 13:00:00,1988-02-15 00:00:00,50,F,M,0,0,300,F,32,5,LIFTING TABLES STRAIN LUMBAR SPINE INJURY,25000,27168.60103
WC1536715,1988-01-02 08:00:00,1988-01-20 00:00:00,19,M,S,0,0,126.45,F,40,5,OVER STRETCHED PULLED MUSCLE LEFT SHOULDER,10000,5337.074832
WC1984019,1988-01-02 11:00:00,1988-03-11 00:00:00,45,M,M,0,0,334.9,F,30,5,STEPPED OFF FORKLIFT NECK,2000,3994.179009
WC2094320,1988-01-03 02:00:00,1988-01-10 00:00:00,34,F,M,0,0,321.7,F,37.25,5,FOREIGN 

3. Utilizamos Pandas poder consultar y tratar más fácilmente el fichero en algunos momentos

In [9]:
import pandas as pd
df = pd.read_csv("/content/train.csv")
df.head(5)

,ClaimNumber,DateTimeOfAccident,DateReported,Age,Gender,MaritalStatus,DependentChildren,DependentsOther,WeeklyWages,PartTimeFullTime,HoursWorkedPerWeek,DaysWorkedPerWeek,ClaimDescription,InitialIncurredCalimsCost,UltimateIncurredClaimCost
0,WC1840818,1988-01-01 09:00:00,1988-03-07 00:00:00,59,F,M,0,0,334.90,F,40.00,5,STRUCK KNIFE SLIPPED CUT FINGER LACERATED RIGH...,2000,1527.825031
1,WC1887818,1988-01-01 09:00:00,1988-01-21 00:00:00,45,F,S,0,0,48.26,F,31.75,5,STRUCK FALLING DOOR PULLED MUSCLE IN FOREARM,2000,49711.115980
2,WC1693016,1988-01-01 13:00:00,1988-02-15 00:00:00,50,F,M,0,0,300.00,F,32.00,5,LIFTING TABLES STRAIN LUMBAR SPINE INJURY,25000,27168.601030
3,WC1536715,1988-01-02 08:00:00,1988-01-20 00:00:00,19,M,S,0,0,126.45,F,40.00,5,OVER STRETCHED PULLED MUSCLE LEFT SHOULDER,10000,5337.074832
4,WC1984019,1988-01-02 11:00:00,1988-03-11 00:00:00,45,M,M,0,0,334.90,F,30.00,5,STEPPED OFF FORKLIFT NECK,2000,3994.179009


Para construir nuestro código Pig será necesario ver que columnas tenemos en el fichero.

In [10]:
df.columns

Index(['ClaimNumber', 'DateTimeOfAccident', 'DateReported', 'Age', 'Gender',
       'MaritalStatus', 'DependentChildren', 'DependentsOther', 'WeeklyWages',
       'PartTimeFullTime', 'HoursWorkedPerWeek', 'DaysWorkedPerWeek',
       'ClaimDescription', 'InitialIncurredCalimsCost',
       'UltimateIncurredClaimCost'],
      dtype='object')

#### Procesar Nulos



1. Comprobamos si hay nulos en nuestro fichero

In [11]:
print(df.shape)
pd.isnull(df).any()

(36176, 15)


ClaimNumber                  False
DateTimeOfAccident           False
DateReported                 False
Age                          False
Gender                       False
MaritalStatus                 True
DependentChildren            False
DependentsOther              False
WeeklyWages                   True
PartTimeFullTime             False
HoursWorkedPerWeek            True
DaysWorkedPerWeek            False
ClaimDescription             False
InitialIncurredCalimsCost    False
UltimateIncurredClaimCost    False
dtype: bool

2. Proceso Pig para sustituir los valores nulos en los campos concretos

In [12]:
%%writefile procesar_nulos.pig

claims = load 'train.csv' using PigStorage(',') as (
    ClaimNumber:chararray, DateTimeOfAccident:chararray, DateReported:chararray, Age:int, Gender:chararray,
       MaritalStatus:chararray, DependentChildren:int, DependentsOther:int, WeeklyWages:float,
       PartTimeFullTime:chararray, HoursWorkedPerWeek:float, DaysWorkedPerWeek:int,
       ClaimDescription:chararray, InitialIncurredCalimsCost:int, UltimateIncurredClaimCost:int);

claims_clean = foreach claims generate ClaimNumber..Gender,
      ((MaritalStatus IS  NULL) ? 'U' : MaritalStatus), DependentChildren, DependentsOther, ((WeeklyWages IS  NULL) ? 0 : WeeklyWages),
      PartTimeFullTime,((HoursWorkedPerWeek IS  NULL) ? 0 : HoursWorkedPerWeek) , DaysWorkedPerWeek..UltimateIncurredClaimCost
    ;


STORE claims_clean INTO 'nulos' using PigStorage(',');

Writing procesar_nulos.pig


In [13]:
!pig procesar_nulos.pig

2024-02-09 08:16:19,731 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-02-09 08:16:19,733 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
2024-02-09 08:16:19,733 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2024-02-09 08:16:19,842 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-02-09 08:16:19,842 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1707466579829.log
2024-02-09 08:16:20,332 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-02-09 08:16:20,445 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-02-09 08:16:20,445 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2024-02-09 08:16:20,495 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the ses

3. Comprobación mediante Python/Pandas

In [14]:
df_clean = pd.read_csv("/content/nulos/part-m-00000", skiprows=1, header = None, names = df.columns)
print(df_clean.shape)
pd.isnull(df_clean).any()


(36176, 15)


ClaimNumber                  False
DateTimeOfAccident           False
DateReported                 False
Age                          False
Gender                       False
MaritalStatus                False
DependentChildren            False
DependentsOther              False
WeeklyWages                  False
PartTimeFullTime             False
HoursWorkedPerWeek           False
DaysWorkedPerWeek            False
ClaimDescription             False
InitialIncurredCalimsCost    False
UltimateIncurredClaimCost    False
dtype: bool

5. Eliminar los nulos (menos filas)

In [15]:
%%writefile filtrar_nulos.pig
claims = load '/content/train.csv' using PigStorage(',') as (
    ClaimNumber:chararray, DateTimeOfAccident:chararray, DateReported:chararray, Age:int, Gender:chararray,
       MaritalStatus:chararray, DependentChildren:int, DependentsOther:int, WeeklyWages:float,
       PartTimeFullTime:chararray, HoursWorkedPerWeek:float, DaysWorkedPerWeek:int,
       ClaimDescription:chararray, InitialIncurredCalimsCost:int, UltimateIncurredClaimCost:int);

claims_clean = FILTER claims BY NOT ((MaritalStatus IS  NULL) OR (HoursWorkedPerWeek IS  NULL) OR (WeeklyWages IS  NULL));


STORE claims_clean INTO 'limpios' using PigStorage(',');

Writing filtrar_nulos.pig


In [16]:
!pig filtrar_nulos.pig

2024-02-09 08:17:00,138 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-02-09 08:17:00,149 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
2024-02-09 08:17:00,149 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2024-02-09 08:17:00,241 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-02-09 08:17:00,241 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1707466620224.log
2024-02-09 08:17:00,746 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-02-09 08:17:00,863 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-02-09 08:17:00,864 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2024-02-09 08:17:00,916 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the ses

In [17]:
df_clean = pd.read_csv("/content/limpios/part-m-00000", skiprows=1, header = None, names = df.columns)
print(df_clean.shape)
pd.isnull(df_clean).any()

(36048, 15)


ClaimNumber                  False
DateTimeOfAccident           False
DateReported                 False
Age                          False
Gender                       False
MaritalStatus                False
DependentChildren            False
DependentsOther              False
WeeklyWages                  False
PartTimeFullTime             False
HoursWorkedPerWeek           False
DaysWorkedPerWeek            False
ClaimDescription             False
InitialIncurredCalimsCost    False
UltimateIncurredClaimCost    False
dtype: bool

#### Procesar Fecha

1. Revisamos los formatos de fecha y vemos que hay un día el 18/01/1988 en el que se grabaron mal las fechas.

In [18]:
!grep "1988/" /content/train.csv

WC1938519,1988-01-07 13:00:00,1988/01/18 ,34,M,S,0,0,400,F,38,5,DROPPED BRICKS ON TROLLEY MUSCLE STRAIN LOWER BACK STRAIN,10000,8731.598915
WC1519515,1988-01-11 20:00:00,1988/01/18 ,39,M,S,0,0,355.7,F,38,5,LOADING BARS ONTO TROLLEY STRAINED BACK,10000,9969.605159
WC2044520,1988-01-14 20:00:00,1988/01/18 ,30,M,M,0,0,352.8,F,40,5,PULLING PALLET TRUCK STRAINED BACK MUSCLE TORN,10000,8997.98091
WC1895018,1988/01/18 08:00:00,1988-01-31 00:00:00,39,F,M,2,1,266,F,40,5,HEAVY LIFTING CONCRETE ETC HERNIA UPPER STOMACH,10000,15554.92106
WC1511415,1988/01/18 09:00:00,1988-02-29 00:00:00,41,M,M,0,0,350,F,40,5,SHOULDER ELBOW INJURY,10000,10455.94582
WC1665716,1988/01/18 ,1988-02-17 00:00:00,20,M,S,0,0,238.8,F,38,5,STRUCK SLEDGE HAMMER LACERATED RIGHT HAND LACERATION RIGHT HAND,2000,3347.906675
WC1671516,1988/01/18 ,1988-02-21 00:00:00,23,M,M,0,0,330,F,40,5,JAMMED RIGHT HAND PUNCTURE WOUND RIGHT ARM,2000,2251.98591
WC1959919,1988/01/18 ,1988-02-07 00:00:00,22,M,S,0,0,340.6,F,38,5,LEVER SLIP THROWN OF

2. Escribimos el script de Pig para tratar esta circunstancia

In [19]:
%%writefile procesar_fechas.pig
a = load '/content/train.csv' using PigStorage(',') as (
    ClaimNumber:chararray, DateTimeOfAccident:chararray, DateReported:chararray, Age:int, Gender:chararray,
       MaritalStatus:chararray, DependentChildren:int, DependentsOther:int, WeeklyWages:float,
       PartTimeFullTime:chararray, HoursWorkedPerWeek:float, DaysWorkedPerWeek:int,
       ClaimDescription:chararray, InitialIncurredCalimsCost:int, UltimateIncurredClaimCost:int);

goodDate = filter a by DateTimeOfAccident matches '\\d+-.+';
parseBadDate = foreach (filter a by DateTimeOfAccident matches '\\d+/.+') generate
    ClaimNumber, REGEX_EXTRACT_ALL(DateTimeOfAccident, '(\\d+)/(\\d+)/(\\d+).*') as datenew:(year, month, day),
     DateReported..UltimateIncurredClaimCost;

fixedBadDate = foreach parseBadDate generate ClaimNumber,
      CONCAT(CONCAT(CONCAT(CONCAT(CONCAT((chararray) datenew.year, '-'), (chararray)datenew.month), '-'), (chararray)datenew.day), ' 00:00:00') as date,
      DateReported..UltimateIncurredClaimCost;

together = union goodDate, fixedBadDate;
STORE together INTO 'goodDates' USING PigStorage(',');

Writing procesar_fechas.pig


3. Ejecutamos almacenando en goodDates

In [20]:
!pig procesar_fechas.pig

2024-02-09 08:17:35,908 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-02-09 08:17:35,915 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
2024-02-09 08:17:35,915 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2024-02-09 08:17:36,036 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-02-09 08:17:36,036 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1707466656018.log
2024-02-09 08:17:36,552 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-02-09 08:17:36,651 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-02-09 08:17:36,651 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2024-02-09 08:17:36,700 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the ses

4. Comprobamos que ya en el campo que hemos modificado la fecha ya está bien puesta, pero aún queda algún registro incorrecto ¿Cómo lo corregiríamos?

In [21]:
!grep "1988/" /content/goodDates/*

/content/goodDates/part-m-00000:WC1938519,1988-01-07 13:00:00,1988/01/18 ,34,M,S,0,0,400.0,F,38.0,5,DROPPED BRICKS ON TROLLEY MUSCLE STRAIN LOWER BACK STRAIN,10000,8731
/content/goodDates/part-m-00000:WC1519515,1988-01-11 20:00:00,1988/01/18 ,39,M,S,0,0,355.7,F,38.0,5,LOADING BARS ONTO TROLLEY STRAINED BACK,10000,9969
/content/goodDates/part-m-00000:WC2044520,1988-01-14 20:00:00,1988/01/18 ,30,M,M,0,0,352.8,F,40.0,5,PULLING PALLET TRUCK STRAINED BACK MUSCLE TORN,10000,8997


#### Unir actualización y marcarlas con flag

**Nota: Generamos previous con los primeros registros y today con los últimos**

In [22]:
df.head(10).to_csv("previous.csv", index = False, header = False)
df.tail(10).to_csv("today.csv", index = False, header = False)

In [ ]:
%%writefile procesar_incremento.pig
today = load 'today.csv' using PigStorage(',') as (
    ClaimNumber:chararray, DateTimeOfAccident:chararray, DateReported:chararray, Age:int, Gender:chararray,
       MaritalStatus:chararray, DependentChildren:int, DependentsOther:int, WeeklyWages:float,
       PartTimeFullTime:chararray, HoursWorkedPerWeek:float, DaysWorkedPerWeek:int,
       ClaimDescription:chararray, InitialIncurredCalimsCost:int, UltimateIncurredClaimCost:int);

todayWithFlag = foreach today generate *, 0 as flag;

previous = load 'previous.csv' using PigStorage(',') as ( ClaimNumber:chararray, DateTimeOfAccident:chararray, DateReported:chararray, Age:int, Gender:chararray,
       MaritalStatus:chararray, DependentChildren:int, DependentsOther:int, WeeklyWages:float,
       PartTimeFullTime:chararray, HoursWorkedPerWeek:float, DaysWorkedPerWeek:int,
       ClaimDescription:chararray, InitialIncurredCalimsCost:int, UltimateIncurredClaimCost:int);

previousWithFlag = foreach previous generate *, 1 as flag;

combined = union todayWithFlag, previousWithFlag;
grouped = group combined by ClaimNumber;
result = foreach grouped {
 a = order combined by flag;
 b = limit a 1;
 generate flatten(b);
}

resultNoFlag = foreach result generate ClaimNumber..UltimateIncurredClaimCost, flag;

STORE resultNoFlag INTO 'result' USING PigStorage (',');

Writing procesar_incremento.pig


In [23]:
!pig procesar_incremento.pig

2024-02-09 08:18:01,521 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-02-09 08:18:01,528 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
2024-02-09 08:18:01,528 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2024-02-09 08:18:01,634 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-02-09 08:18:01,634 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1707466681619.log
2024-02-09 08:18:02,226 [main] ERROR org.apache.pig.Main - ERROR 2997: Encountered IOException. File procesar_incremento.pig does not exist
Details at logfile: /content/pig_1707466681619.log
2024-02-09 08:18:02,260 [main] INFO  org.apache.pig.Main - Pig script completed in 843 milliseconds (843 ms)


In [24]:
!hdfs dfs -cat result/*

cat: `result/*': No such file or directory


#### Procesar cadenas de caracteres

In [25]:
%%writefile procesar_descriptions.pig

claims = LOAD 'today.csv' using PigStorage(',') as (
    ClaimNumber:chararray, DateTimeOfAccident:chararray, DateReported:chararray, Age:int, Gender:chararray,
       MaritalStatus:chararray, DependentChildren:int, DependentsOther:int, WeeklyWages:float,
       PartTimeFullTime:chararray, HoursWorkedPerWeek:float, DaysWorkedPerWeek:int,
       ClaimDescription:chararray, InitialIncurredCalimsCost:int, UltimateIncurredClaimCost:int);

-- transformamos la cadena en tupla (no flatten)
claims_tokens = FOREACH claims GENERATE ClaimNumber..DaysWorkedPerWeek,
       TOKENIZE(ClaimDescription), InitialIncurredCalimsCost, UltimateIncurredClaimCost;

DUMP claims_tokens;


Writing procesar_descriptions.pig


In [26]:
!pig procesar_descriptions.pig

2024-02-09 08:18:34,509 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-02-09 08:18:34,516 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
2024-02-09 08:18:34,516 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2024-02-09 08:18:34,639 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-02-09 08:18:34,639 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1707466714620.log
2024-02-09 08:18:35,207 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-02-09 08:18:35,299 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-02-09 08:18:35,299 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2024-02-09 08:18:35,351 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the ses

In [ ]:
%%writefile wordcounts_descriptions.pig

claims = LOAD '/content/train.csv' using PigStorage(',') as (
    ClaimNumber:chararray, DateTimeOfAccident:chararray, DateReported:chararray, Age:int, Gender:chararray,
       MaritalStatus:chararray, DependentChildren:int, DependentsOther:int, WeeklyWages:float,
       PartTimeFullTime:chararray, HoursWorkedPerWeek:float, DaysWorkedPerWeek:int,
       ClaimDescription:chararray, InitialIncurredCalimsCost:int, UltimateIncurredClaimCost:int);

-- Tokenizer/Aplanar Descriptions
wordfile_flat = FOREACH claims GENERATE FLATTEN (TOKENIZE(ClaimDescription)) as wordin;
-- Agrupación por palabras (GROUP BY)
wordfile_grpd = GROUP wordfile_flat by wordin;
word_counts = FOREACH wordfile_grpd GENERATE group, COUNT(wordfile_flat.wordin) as cnt;
-- filtrar palabras demasiado comunes
word_counts = FILTER word_counts BY ($0 != 'RIGHT' AND $0 != 'LEFT' AND $0 != 'AND' AND $0 != 'ON');
-- ordenar
word_count_des = ORDER word_counts BY cnt DESC;
DUMP word_count_des;


Overwriting wordcounts_descriptions.pig


In [ ]:
!pig wordcounts_descriptions.pig

2023-03-03 17:37:41,504 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2023-03-03 17:37:41,507 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
2023-03-03 17:37:41,507 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2023-03-03 17:37:41,633 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2023-03-03 17:37:41,633 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1677865061606.log
2023-03-03 17:37:42,062 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2023-03-03 17:37:42,188 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2023-03-03 17:37:42,188 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2023-03-03 17:37:42,232 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the ses

### Otras transformaciones interesantes

Pivotado:
- [De columnas a filas](https://stackoverflow.com/questions/11213567/pivot-table-with-apache-pig)
- [De filas a columnas](https://stackoverflow.com/questions/11578815/pivoting-in-pig)


[Completar fechas faltantes](https://stackoverflow.com/questions/17931813/how-to-create-missing-records-within-date-time-range-in-pig-latin)

[Borrar Duplicados](https://stackoverflow.com/questions/11534041/removing-duplicates-using-piglatin)

[Transformar Columnas en Filas](https://stackoverflow.com/questions/18010826/in-apache-pig-how-can-i-serialise-columns-into-rows)

[Dividir una tupla](https://stackoverflow.com/questions/11287362/splitting-a-tuple-into-multiple-tuples-in-pig)

[Lecturas de csv sin header](
https://stackoverflow.com/questions/19115298/skipping-the-header-while-loading-the-text-file-using-piglatin)